In [1]:
!wget https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020
!apt-key add GPG-PUB-KEY-INTEL-OPENVINO-2020
!touch /etc/apt/sources.list.d/intel-openvino-2020.list
!echo "deb https://apt.repos.intel.com/openvino/2020 all main" >> /etc/apt/sources.list.d/intel-openvino-2020.list

!apt update
!apt install intel-openvino-dev-ubuntu18-2020.4.287

!pip install test-generator==0.1.1
!source /opt/intel/openvino/bin/setupvars.sh

--2021-06-21 05:26:05--  https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020
Resolving apt.repos.intel.com (apt.repos.intel.com)... 104.94.124.128, 2600:1408:2000:690::4b23, 2600:1408:2000:686::4b23
Connecting to apt.repos.intel.com (apt.repos.intel.com)|104.94.124.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 939 [binary/octet-stream]
Saving to: ‘GPG-PUB-KEY-INTEL-OPENVINO-2020’

GPG-PUB-KEY-INTEL-O 100%[===================>]     939  --.-KB/s    in 0s      

2021-06-21 05:26:06 (43.9 MB/s) - ‘GPG-PUB-KEY-INTEL-OPENVINO-2020’ saved [939/939]

OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 https://apt.repos.intel.com/openvino/2020 all InRelease [5,659 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
I

In [2]:
!pip install gdown
!gdown --id 1j9x9PX8YIrWXlv9GVE68c8jUiM8MexKA
!unzip mangodetection.zip
!mv /content/content/mangodetection /content/

Downloading...
From: https://drive.google.com/uc?id=1j9x9PX8YIrWXlv9GVE68c8jUiM8MexKA
To: /content/mangodetection.zip
767MB [00:04, 191MB/s]
Archive:  mangodetection.zip
   creating: content/mangodetection/
   creating: content/mangodetection/test/
   creating: content/mangodetection/test/healthy/
  inflating: content/mangodetection/test/healthy/0001_0137.JPG  
  inflating: content/mangodetection/test/healthy/0001_0072.JPG  
  inflating: content/mangodetection/test/healthy/0001_0152.JPG  
  inflating: content/mangodetection/test/healthy/0001_0092.JPG  
  inflating: content/mangodetection/test/healthy/0001_0004.JPG  
  inflating: content/mangodetection/test/healthy/0001_0051.JPG  
  inflating: content/mangodetection/test/healthy/0001_0097.JPG  
  inflating: content/mangodetection/test/healthy/0001_0105.JPG  
  inflating: content/mangodetection/test/healthy/0001_0012.JPG  
   creating: content/mangodetection/test/diseased/
  inflating: content/mangodetection/test/diseased/0012_0210.JPG  

In [3]:
import tensorflow as tf
import IPython.display as display
import PIL
import PIL.Image
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [4]:
data_dir = pathlib.Path('/content/mangodetection/train')
test_dir = pathlib.Path('/content/mangodetection/test')
train_count = len(list(data_dir.glob('*/*.JPG')))
test_count = len(list(test_dir.glob('*/*.JPG')))

batch_size = 32
img_height = 180
img_width = 180

#Generate More Data by Generator
image_gen_train = ImageDataGenerator(rescale = 1./255,validation_split=0.2,rotation_range = 45,width_shift_range=.15,height_shift_range =.15,horizontal_flip=True,zoom_range=0.5)

#Shuffle All Data 
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'training',  directory= data_dir,shuffle=True,target_size=(img_height, img_width))  

#Divide Data for Validation
validation_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'validation',  directory= data_dir,shuffle=True,target_size=(img_height, img_width)) 

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])

Found 330 images belonging to 2 classes.
Found 82 images belonging to 2 classes.


In [5]:
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model

In [6]:
# Get the ResNet50 base model
basemodel = ResNet50(weights = 'imagenet', include_top = False, input_tensor = Input(shape=(180, 180, 3)))

94773248/94765736 [==============================] - 1s 0us/step


In [7]:
for layer in basemodel.layers:
  layers.trainable = False

In [8]:
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (4,4))(headmodel)
headmodel = Flatten(name= 'flatten')(headmodel)
headmodel = Dense(256, activation = "relu")(headmodel)
headmodel = Dropout(0.3)(headmodel)#
headmodel = Dense(256, activation = "relu")(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(256, activation = "relu")(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(2, activation = 'softmax')(headmodel)

model = Model(inputs = basemodel.input, outputs = headmodel)

In [9]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 180, 180, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 186, 186, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 90, 90, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 90, 90, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [10]:
rlr = ReduceLROnPlateau(patience=10, verbose=1)
es = EarlyStopping(patience=24, restore_best_weights=True, verbose=1)
mc = ModelCheckpoint('/content/mangodetection/model.h5', save_best_only=True, verbose=1)

epochs = 15
history = model.fit(
  train_data_gen,
  callbacks=[rlr, es, mc],  
  validation_data=validation_data_gen,
  epochs=15
)

Epoch 1/15
11/11 [==============================] - 161s 11s/step - loss: 0.4129 - accuracy: 0.8152 - val_loss: 321.7531 - val_accuracy: 0.6098

Epoch 00001: val_loss improved from inf to 321.75314, saving model to /content/mangodetection/model.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/15
11/11 [==============================] - 115s 11s/step - loss: 0.1475 - accuracy: 0.9485 - val_loss: 57.4949 - val_accuracy: 0.6098

Epoch 00002: val_loss improved from 321.75314 to 57.49485, saving model to /content/mangodetection/model.h5
Epoch 3/15
11/11 [==============================] - 115s 11s/step - loss: 0.0571 - accuracy: 0.9758 - val_loss: 481.2355 - val_accuracy: 0.3902

Epoch 00003: val_loss did not improve from 57.49485
Epoch 4/15
11/11 [==============================] - 113s 10s/step - loss: 0.1946 - accuracy: 0.9455 - val_loss: 1665.3647 - val_accuracy: 0.3902

Epoch 00004: val_loss did not improve from 57.49485
Epoch 5/15
11/11 [==============================] - 113s 10s/step - loss: 0.1981 - accuracy: 0.9273 - val_loss: 2552.7737 - val_accuracy: 0.6098

Epoch 00005: val_loss did not improve from 57.49485
Epoch 6/15
11/11 [==============================] - 113s 10s/step - loss: 0.0670 - accuracy: 0.9879 - val_loss: 65.5352 - val_accuracy: 0.6098

Epoch 00006

In [11]:
model_mango = load_model('/content/mangodetection/model.h5')

image_gen_test = ImageDataGenerator(rescale = 1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=32,directory= test_dir,shuffle=True,target_size=(img_height, img_width))  

start_time = time.time()
predictions = model_mango.predict(test_data_gen)
for i in range(len(predictions)):
    if(predictions[i][0]<0):
      print("Healthy     "+str(abs(predictions[i][0]))+"%")
    else:  
      print("Diseased    "+str(100-abs(predictions[i][0]))+"%")
print("Total Time: ", time.time()-start_time)
print("Average Time Per Image: ", (time.time()-start_time)/len(predictions))


Found 23 images belonging to 2 classes.
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Diseased    99.0%
Total Time:  13.992429494857788
Average Time Per Image:  0.6083720766979716


In [12]:
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
import tensorflow.compat.v1 as tf
#Using Tensorflow 1.0 due to Compatibility issues
tf.disable_v2_behavior()

# Clear any previous session.
tf.keras.backend.clear_session()

#Directory for Saving Model
save_pb_dir = '/content/mangodetection'
model_fname = '/content/mangodetection/model.h5'

def freeze_graph(graph, session, output, save_pb_dir='/content/mangodetection', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.compat.v1.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.compat.v1.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen


tf.keras.backend.set_learning_phase(0) 

model = load_model(model_fname)
session=tf.keras.backend.get_session()
INPUT_NODE = [t.op.name for t in model.inputs]
OUTPUT_NODE = [t.op.name for t in model.outputs]
print(INPUT_NODE, OUTPUT_NODE)
frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

Instructions for updating:
non-resource variables are not supported in the long term


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:435: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


['input_1'] ['dense_3/Softmax']
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [13]:

mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'

pb_file = '/content/mangodetection/frozen_model.pb'
output_dir = '/content/mangodetection'
img_height = 180
input_shape = [1, img_height, img_height, 3]
input_shape_str = str(input_shape).replace(' ', '')

#Running Commands to run OPENVINO's Model Optimiser Converter 'mo_tf.py'
!source /opt/intel/openvino/bin/setupvars.sh
!python {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP16

[setupvars.sh] OpenVINO environment initialized
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/mangodetection/frozen_model.pb
	- Path for generated IR: 	/content/mangodetection
	- IR output name: 	frozen_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,180,180,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output nod

In [14]:
!source /opt/intel/openvino/bin/setupvars.sh && \
python /content/mangodetection/inference.py

[setupvars.sh] OpenVINO environment initialized
2021-06-21 05:58:11.552816: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/content/mangodetection/inference.py:37: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  plugin = IEPlugin("CPU", plugin_dirs=plugin_dir)
/content/mangodetection/inference.py:39: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  net = IENetwork(model=model_xml, weights=model_bin)
/content/mangodetection/inference.py:40: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  assert len(net.inputs.keys()) == 1
Traceback (most recent call last):
  File "/content/mangodetection/inference.py", line 68, in <module>
    if(predictions['dense_1/BiasAdd

In [ ]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("SNO5N6UNOGXMICYSINU4DIEJ-AFBDS2UOAU7ZPMEU")
import anvil.media
import numpy as np

from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array

import cv2
%cd /content
!mkdir tst
%cd tst
!mkdir test
%cd /content
@anvil.server.callable
def imgload(file):
  model_mango = load_model('/content/mangodetection/model.h5')
  with anvil.media.TempFile(file) as filename:
    img = load_img(filename)
    img_array = img_to_array(img)
    save_img('/content/tst/test/temp.png',img_array)
    print("Processed Incoming Image")
    
  image_gen_test = ImageDataGenerator(rescale = 1./255)
  test_data_gen = image_gen_test.flow_from_directory(batch_size=1,directory= "/content/tst",shuffle=True,target_size=(img_height, img_width)) 
  predictions = model_mango.predict(test_data_gen) 
  print(test_data_gen,predictions)
  li=''
  if(predictions[0][0]<0):
      li='Healthy'
  else:  
    li="Diseased"
  return(li)
anvil.server.wait_forever()  

     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp37-none-any.whl size=45230 sha256=d88a5877e205bac14e04c333f17c2dd6c9970afcccba0b3f11530038f781cc54
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER
/content
/content/tst
/content
